In [10]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [11]:
ticker = 'QQQ'
min_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]

df_daily, feature_sets, return_cols, daily_cols, sets, set_names, feature_cols = train_flow.import_data(ticker, min_feats, returns) 

# Add any new features
#df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
df_main = df_daily.copy()#[df_daily['Date'] <= '2026-01-21'].copy()

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope'])


# Run Train_Flow

In [19]:
# -----------------------------
# Handling of new features and permutation importance
# -----------------------------
pi_handlings = ['include_new'] #'exclude_new', 'include_new', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
new_features = [] #past_sum_cols

# -----------------------------
# Run grid (feature sets x horizon x train_years, etc.)
# -----------------------------
target_horizons = [2, 10, 30]#[2, 5, 10, 20, 30]
train_years = [5]
pi_years = [1.5]
min_feats = [8]
days_assessed = 0
test_day = 1
feature_cols = feature_cols
models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
list_name = 'kitchen_sink'

results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                   train_years, pi_years, min_feats, list_name, new_features)
results_df

Running for horizon 2 | include_new
Running for horizon 10 | include_new
Running for horizon 30 | include_new


,feature_set,horizon
